#Data Engineering Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
! pip install utils
import numpy as np
import pylab
import os, sys
import pandas as pd
import io
import requests
import warnings
import utils

Now that the funtions have been imported the team can focus on the download coding. The following cells will set up an install for Kaggle files and prompt for an upload of the kaggle.json file for credentials. 

The mkdir function creates a directory for the Kaggle data. This cell will allow the team to verify that the kaggle.json file appropriately uploaded to the directory.

In [2]:
##import the kaggle.json from local to drive
!pip install -q kaggle
from google.colab import files
##when it asks you to choose a file select the kaggle.json located within the 'project' folder from the github repo
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chelseagorius","key":"0a34819ed937ff55d31f4288ab40cf19"}'}

In [3]:
##make a kaggle and a data folder
!mkdir ~/.kaggle
!mkdir data
##copy the kaggle.json to the .kaggle folder then grant permissions
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#test to see if kaggle is working, should print list of datasets
!kaggle datasets list

ref                                                       title                                         size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
manchunhui/us-election-2020-tweets                        US Election 2020 Tweets                      326MB  2020-11-08 12:37:15            894  
unanimad/us-election-2020                                 US Election 2020                             418KB  2020-11-08 13:51:30            917  
headsortails/us-election-2020-presidential-debates        US Election 2020 - Presidential Debates      199MB  2020-10-23 16:56:10            257  
radustoicescu/2020-united-states-presidential-election    2020 United States presidential election      11MB  2019-07-04 15:00:45            578  
etsc9287/2020-general-election-polls                      2020 General Election Polls                  109KB  2020-02-

Now, the team must download all of the datasets for the class. The three datasets are focused on the NBA. 

The first dataset is for injuries. Each injury will be used to set up players, timeframes, and severity of injuries. 

The other two datasets are for the player performance. By cross referencing this data to the previous list, the team will be able to see which players are limited from the injury and how performance is hampered by time in rehab.

In [4]:
##downloading all the datasets
!kaggle datasets download -d ghopkins/nba-injuries-2010-2018
!kaggle datasets download -d nathanlauga/nba-games
!kaggle datasets download -d pablote/nba-enhanced-stats

  0% 0.00/226k [00:00<?, ?B/s]
100% 226k/226k [00:00<00:00, 83.0MB/s]
 50% 9.00M/18.1M [00:00<00:00, 25.2MB/s]
100% 18.1M/18.1M [00:00<00:00, 45.8MB/s]
 54% 9.00M/16.7M [00:01<00:00, 10.4MB/s]
100% 16.7M/16.7M [00:01<00:00, 16.4MB/s]


In [5]:
##unzipping to the data folder
!unzip nba-injuries-2010-2018.zip -d data
!unzip nba-games.zip -d data
!unzip nba-enhanced-stats.zip -d data

Archive:  nba-injuries-2010-2018.zip
  inflating: data/injuries_2010-2020.csv  
Archive:  nba-games.zip
  inflating: data/games.csv          
  inflating: data/games_details.csv  
  inflating: data/players.csv        
  inflating: data/ranking.csv        
  inflating: data/teams.csv          
Archive:  nba-enhanced-stats.zip
  inflating: data/2012-18_officialBoxScore.csv  
  inflating: data/2012-18_playerBoxScore.csv  
  inflating: data/2012-18_standings.csv  
  inflating: data/2012-18_teamBoxScore.csv  
  inflating: data/2016-17_officialBoxScore.csv  
  inflating: data/2016-17_playerBoxScore.csv  
  inflating: data/2016-17_standings.csv  
  inflating: data/2016-17_teamBoxScore.csv  
  inflating: data/2017-18_officialBoxScore.csv  
  inflating: data/2017-18_playerBoxScore.csv  
  inflating: data/2017-18_standings.csv  
  inflating: data/2017-18_teamBoxScore.csv  
  inflating: data/metadata_officialBoxScore.pdf  
  inflating: data/metadata_playerBoxScore.pdf  
  inflating: data/metadata

The team must now use these downloads to create dataframes. Pandas dataframes will be easier to manage the data. The team will be able to use Pandas to process the data and allow the team to make correlations for feature engineering to create the models.

In [6]:
#create a list for each data set
ds_NBA_Injuries, ds_NBA_Games, ds_NBA_Enhanced = [], [], []

#import csv files as dataframes and save to respective list, injury set first
df_Injuries = pd.read_csv('data/injuries_2010-2020.csv')
df_Injury_Start = df_Injuries[df_Injuries.Acquired.isnull()]
df_Injury_End = df_Injuries[df_Injuries.Relinquished.isnull()]
ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('data/games.csv')
df_Games_gamesDetails = pd.read_csv('data/games_details.csv')
df_Games_players = pd.read_csv('data/players.csv')
df_Games_ranking = pd.read_csv('data/ranking.csv')
df_Games_teams = pd.read_csv('data/teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]
#nba enhanced stats dataset
df_En_officialBS_1218 = pd.read_csv('data/2012-18_officialBoxScore.csv')
df_En_playerBS_1218 = pd.read_csv('data/2012-18_playerBoxScore.csv')
df_En_standings_1218 = pd.read_csv('data/2012-18_standings.csv')
df_En_teamBS_1218 = pd.read_csv('data/2012-18_teamBoxScore.csv')  
df_En_officialBS_1617 = pd.read_csv('data/2016-17_officialBoxScore.csv')  
df_En_playerBS_1617 = pd.read_csv('data/2016-17_playerBoxScore.csv')
df_En_standings_1617 = pd.read_csv('data/2016-17_standings.csv')
df_En_teamBS_1617 = pd.read_csv('data/2016-17_teamBoxScore.csv')  
df_En_officialBS_1718 = pd.read_csv('data/2017-18_officialBoxScore.csv')  
df_En_playerBS_1718 = pd.read_csv('data/2017-18_playerBoxScore.csv')
df_En_standings_1718 = pd.read_csv('data/2017-18_standings.csv')
df_En_teamBS_1718 = pd.read_csv('data/2017-18_teamBoxScore.csv')  
##data/metadata_officialBoxScore.pdf, data/metadata_playerBoxScore.pdf, data/metadata_standing.pdf, data/metadata_teamBoxScore.pdf  
df_En_teamBS = pd.read_csv('data/teamBoxScore.csv')
ds_NBA_Enhanced = [df_En_officialBS_1218, df_En_officialBS_1617, df_En_officialBS_1718, df_En_playerBS_1218, df_En_playerBS_1617, df_En_playerBS_1718, df_En_standings_1218, df_En_standings_1617, df_En_standings_1718, \
                       df_En_teamBS_1218, df_En_teamBS_1617, df_En_teamBS_1718, df_En_teamBS]


#probably need some more data exploration and some feature engineering

Preparing data tables to have the appropriate columns in order to calculate time and player specific metrics for each injury.

In [7]:
#Create a dataframe of distinct player and player IDs
df_distinct_playerID = df_Games_players[["PLAYER_NAME", "PLAYER_ID"]].drop_duplicates()
df_distinct_playerID.astype({'PLAYER_ID':'object'}).dtypes
#Create a dataframe of distinct gameID and game dates
df_Games_games['GAME_DATE_EST'] = pd.to_datetime(df_Games_games['GAME_DATE_EST'])
df_distinct_gameId_date = df_Games_games[["GAME_ID", "GAME_DATE_EST"]].drop_duplicates()
#Join the distinct player df by player name and join the gameID information
df_Injury_Start = df_Injury_Start.join(df_distinct_playerID.astype('object').set_index('PLAYER_NAME'), on='Relinquished')
df_Injury_Start = df_Injury_Start.merge(df_Games_teams[["TEAM_ID", "NICKNAME"]], left_on="Team", right_on="NICKNAME")
df_Injury_Start.drop(['NICKNAME'], axis=1)#.apply(lambda x: x.date())
df_Injury_Start['Date']= pd.to_datetime(df_Injury_Start['Date'])
#Do the same for the Injury End database
df_Injury_End = df_Injury_End.join(df_distinct_playerID.astype('object').set_index('PLAYER_NAME'), on='Acquired')
df_Injury_End = df_Injury_End.merge(df_Games_teams[["TEAM_ID", "NICKNAME"]], left_on="Team", right_on="NICKNAME")
df_Injury_End.drop(['NICKNAME'], axis=1)#.apply(lambda x: x.date())
df_Injury_End['Date']= pd.to_datetime(df_Injury_End['Date'])
# df_distinct_playerID=df_distinct_playerID.sort_values('PLAYER_NAME')
df_Games_gamesDetails = df_Games_gamesDetails.merge(df_distinct_gameId_date, on="GAME_ID")
#graph looking at difference in stats and injury length
##calculating injury length could be hard
df_Injury_length = df_Injury_Start.copy()
df_Injury_length = df_Injury_length.rename(columns={"Date":"DateInjured", "Relinquished":"Player", "Notes":"InjuryNotes"})
df_Injury_length = df_Injury_length[["TEAM_ID", "Team", "PLAYER_ID", "Player", "DateInjured", "InjuryNotes"]]

Transforming the minutes column to a numeric value that can be used to create calculated metrics.

In [114]:
for index, row in df_Games_gamesDetails.iterrows():
  try:
    m, s = str(row.MIN).split(':')
  except (SyntaxError, ValueError) as e:
    m = (row.MIN)
    s = 0
  df_Games_gamesDetails.loc[index,'MIN'] = pd.to_numeric(m) + pd.to_numeric(s)/60
df_Games_gamesDetails.to_csv('df_Games_gamesDetails.csv')

In [122]:
#df_Games_gamesDetails = pd.read_csv('df_Games_gamesDetails.csv')
#df_Games_gamesDetails['GAME_DATE_EST'] = pd.to_datetime(df_Games_gamesDetails['GAME_DATE_EST'])

Creating db that contains the injury start and end date for each injury listed in the original db.

In [107]:
df_Injury_length = df_Injury_length[df_Injury_length['Player'] != np.nan]
inj_count = df_Injury_length['Player'].value_counts()
#First find the date recovered
for index, row in df_Injury_length.iterrows():
  #get rows with same player ID
  temp = df_Injury_End.loc[df_Injury_End['PLAYER_ID'] == row.PLAYER_ID]
  #get rows after the injury date
  temp2 = temp.loc[(temp['Date'] > row.DateInjured)]
  #get the row with the oldest (smallest) date
  recover = temp2.nsmallest(1, 'Date')
  try:
    df_Injury_length.at[index, 'DateRecovered'] = pd.Series(recover[['Date']].Date).values[0]
    df_Injury_length.at[index, 'RecoverNotes'] = pd.Series(recover[['Notes']].Notes).values[0]
  except (IndexError) as e:
    df_Injury_length.at[index, 'DateRecovered'] = np.nan
    df_Injury_length.at[index, 'RecoverNotes'] = np.nan
  #Get number of injuries
  count_name = row['Player']
  try:
   df_Injury_length.at[index,'NumberInjuries'] = inj_count[count_name]
  except (KeyError) as e:
   pass#print(inj_count[count_name])
#Group by player Id and date recovered to avoid miscount of injuries
group = df_Injury_length.groupby(['PLAYER_ID','DateRecovered'])['DateInjured'].min().reset_index()
df_Injury_length = pd.merge(group, df_Injury_length,  how='left', left_on=['PLAYER_ID','DateRecovered', 'DateInjured'], right_on = ['PLAYER_ID','DateRecovered', 'DateInjured'])
#Calculating injury length in days
df_Injury_length['InjuryLengthDays'] = df_Injury_length['DateRecovered'] - df_Injury_length['DateInjured']
for index, row in df_Injury_length.iterrows():
  df_Injury_length.at[index, 'InjuryLengthDays'] = (row.InjuryLengthDays)
#Saving to a .csv file
df_Injury_length.to_csv('df_Injury_length.csv')

Creating the metrics for player performance metrics during the injury game and summarized for the 5 games prior to the injury.
Creating the metrics for player performance metrics in the first game back from injury and summarized for the 5 games after thedf_Injury_stats.to_csv('df_Injury_stat.csv')

In [146]:
df_Injury_stats = df_Injury_length.copy()
for index, row in df_Injury_stats.iterrows():
  #games of just that player
  plyr = df_Games_gamesDetails.loc[df_Games_gamesDetails['PLAYER_ID'] == row.PLAYER_ID]
  #games before and inlucding injury date#5 games prior and the game of injury, for some reason we need to have 4 different variabels, did not work with resetting the variable 'game_set' to itself
  temp = plyr.loc[(plyr['GAME_DATE_EST'] <= row.DateInjured)]
  inj_gameset = temp.nlargest(6, 'GAME_DATE_EST')
  #games after and inlucding recover date
  temp2 = plyr.loc[(plyr['GAME_DATE_EST'] >= row.DateRecovered)]
  #5 games after and the game of injury, for some reason we need to have 3 different variabels, did not work with resetting the variable 'game_set' to itself
  rev_gameset = temp.nsmallest(6, 'GAME_DATE_EST')
  ##
  ##Start calculated columns for injury games and prior
  if len(inj_gameset) > 0:
    #injury game
    inj_game = inj_gameset.iloc[0]
    #5 games prior to injury
    prior5 = inj_gameset.iloc[1:]
    #storing game data from injury game
    df_Injury_stats.at[index, 'inj_MIN'] = inj_game[['MIN']].MIN
    df_Injury_stats.at[index,'inj_FGA'] = inj_game[['FGA']].FGA
    df_Injury_stats.at[index,'inj_FG_PCT'] = inj_game[['FG_PCT']].FG_PCT
    df_Injury_stats.at[index,'inj_FG3A'] = inj_game[['FG3A']].FG3A
    df_Injury_stats.at[index,'inj_FG3_PCT'] = inj_game[['FG3_PCT']].FG3_PCT
    df_Injury_stats.loc[index,'inj_FTA'] = inj_game[['FTA']].FTA
    df_Injury_stats.loc[index,'inj_FT_PCT'] = inj_game[['FT_PCT']].FT_PCT
    df_Injury_stats.loc[index,'inj_REB'] = inj_game[['REB']].REB
    df_Injury_stats.loc[index,'inj_AST'] = inj_game[['AST']].AST
    df_Injury_stats.loc[index,'inj_STL'] = inj_game[['STL']].STL
    df_Injury_stats.loc[index,'inj_BLK'] = inj_game[['BLK']].BLK
    df_Injury_stats.loc[index,'inj_TO'] = inj_game[['TO']].TO
    df_Injury_stats.loc[index,'inj_PF'] = inj_game[['PF']].PF
    df_Injury_stats.loc[index,'inj_PTS'] = inj_game[['PTS']].PTS
    df_Injury_stats.loc[index,'inj_PLUS_MINUS'] = inj_game[['PLUS_MINUS']].PLUS_MINUS
#storing game data from prior 5 games
    df_Injury_stats.at[index,'p5_MIN'] = prior5[['MIN']].MIN.mean()
    df_Injury_stats.at[index,'p5_FGA'] = prior5[['FGA']].FGA.mean()
    df_Injury_stats.at[index,'p5_FG_PCT'] = prior5[['FG_PCT']].FG_PCT.mean()
    df_Injury_stats.at[index,'p5_FG3A'] = prior5[['FG3A']].FG3A.mean()
    df_Injury_stats.at[index,'p5_FG3_PCT'] = prior5[['FG3_PCT']].FG3_PCT.mean()
    df_Injury_stats.at[index,'p5_FTA'] = prior5[['FTA']].FTA.mean()
    df_Injury_stats.at[index,'p5_FT_PCT'] = prior5[['FT_PCT']].FT_PCT.mean()
    df_Injury_stats.at[index,'p5_REB'] = prior5[['REB']].REB.mean()
    df_Injury_stats.at[index,'p5_AST'] = prior5[['AST']].AST.mean()
    df_Injury_stats.at[index,'p5_STL'] = prior5[['STL']].STL.mean()
    df_Injury_stats.at[index,'p5_BLK'] = prior5[['BLK']].BLK.mean()
    df_Injury_stats.at[index,'p5_TO'] = prior5[['TO']].TO.mean()
    df_Injury_stats.at[index,'p5_PF'] = prior5[['PF']].PF.mean()
    df_Injury_stats.at[index,'p5_PTS'] = prior5[['PTS']].PTS.mean()
    df_Injury_stats.at[index,'p5_PLUS_MINUS'] = prior5[['PLUS_MINUS']].PLUS_MINUS.mean()
  ##
  ##Star calculated column for recovery game and after
  if len(rev_gameset) > 0:
    #injury game
    rev_game = rev_gameset.iloc[0]
    #5 games post injury
    post5 = rev_gameset.iloc[1:]
    #storing game data from injury game
    df_Injury_stats.at[index, 'inj_MIN'] = rev_game[['MIN']].MIN
    df_Injury_stats.at[index,'inj_FGA'] = rev_game[['FGA']].FGA
    df_Injury_stats.at[index,'inj_FG_PCT'] = rev_game[['FG_PCT']].FG_PCT
    df_Injury_stats.at[index,'inj_FG3A'] = rev_game[['FG3A']].FG3A
    df_Injury_stats.at[index,'inj_FG3_PCT'] = rev_game[['FG3_PCT']].FG3_PCT
    df_Injury_stats.loc[index,'inj_FTA'] = rev_game[['FTA']].FTA
    df_Injury_stats.loc[index,'inj_FT_PCT'] = rev_game[['FT_PCT']].FT_PCT
    df_Injury_stats.loc[index,'inj_REB'] = rev_game[['REB']].REB
    df_Injury_stats.loc[index,'inj_AST'] = rev_game[['AST']].AST
    df_Injury_stats.loc[index,'inj_STL'] = rev_game[['STL']].STL
    df_Injury_stats.loc[index,'inj_BLK'] = rev_game[['BLK']].BLK
    df_Injury_stats.loc[index,'inj_TO'] = rev_game[['TO']].TO
    df_Injury_stats.loc[index,'inj_PF'] = rev_game[['PF']].PF
    df_Injury_stats.loc[index,'inj_PTS'] = rev_game[['PTS']].PTS
    df_Injury_stats.loc[index,'inj_PLUS_MINUS'] = rev_game[['PLUS_MINUS']].PLUS_MINUS
    #storing game data from prior 5 games
    df_Injury_stats.at[index,'p5_MIN'] = post5[['MIN']].MIN.mean()
    df_Injury_stats.at[index,'p5_FGA'] = post5[['FGA']].FGA.mean()
    df_Injury_stats.at[index,'p5_FG_PCT'] = post5[['FG_PCT']].FG_PCT.mean()
    df_Injury_stats.at[index,'p5_FG3A'] = post5[['FG3A']].FG3A.mean()
    df_Injury_stats.at[index,'p5_FG3_PCT'] = post5[['FG3_PCT']].FG3_PCT.mean()
    df_Injury_stats.at[index,'p5_FTA'] = post5[['FTA']].FTA.mean()
    df_Injury_stats.at[index,'p5_FT_PCT'] = post5[['FT_PCT']].FT_PCT.mean()
    df_Injury_stats.at[index,'p5_REB'] = post5[['REB']].REB.mean()
    df_Injury_stats.at[index,'p5_AST'] = post5[['AST']].AST.mean()
    df_Injury_stats.at[index,'p5_STL'] = post5[['STL']].STL.mean()
    df_Injury_stats.at[index,'p5_BLK'] = post5[['BLK']].BLK.mean()
    df_Injury_stats.at[index,'p5_TO'] = post5[['TO']].TO.mean()
    df_Injury_stats.at[index,'p5_PF'] = post5[['PF']].PF.mean()
    df_Injury_stats.at[index,'p5_PTS'] = post5[['PTS']].PTS.mean()
    df_Injury_stats.at[index,'p5_PLUS_MINUS'] = post5[['PLUS_MINUS']].PLUS_MINUS.mean()
        #print(inj_game)
df_Injury_stats.to_csv('df_Injury_stat.csv')


#TEST